<a href="https://colab.research.google.com/github/cheul0518/DeepLearning/blob/main/BME595/Homework01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Part A

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import PIL
from PIL import Image
from torchvision import transforms


class Conv2D(nn.Module):
      def __init__(self, in_channel: int, o_channel: int, kernel_size: int, stride: int, mode: str):
          super().__init__()
          self.in_channel = in_channel
          self.o_channel = o_channel
          self.kernel_size = kernel_size
          self.stride = stride
          self.mode = mode
          self.repos = []

      def forward(self, input_image: PIL):
          # Do convolution computation: size of height/width: (H – F + 2P) / S + 1
          # Transform PIL (image) to Tensor (image)
          transform = transforms.Compose([transforms.ToTensor()])
          img = transform(input_image)
          # Height, Width, after convolution
          s = self.stride
          c = self.o_channel # channels
          h = int((img.shape[1] - self.kernel_size + 2*0) / self.stride + 1) # height
          w = int((img.shape[2] - self.kernel_size + 2*0) / self.stride + 1) # width
          conv = torch.zeros(c, h, w)
          for o in range(self.o_channel):
              for i in range(conv.shape[0]):
                  for j in range(conv.shape[1]):
                      for k in range(conv.shape[2]):
                          conv[o,j,k] = conv[o,j,k] + (img[i, j*s:j*s+self.kernel()[o].shape[0], k*s:k*s+self.kernel()[o].shape[1]] * self.kernel()[o]).sum()
          
          # return conv
          #print(F.conv2d(input=img, weight=self.kernel(), stride=1))
          #print(img.shape)
          a = torch.reshape(self.kernel(),(3,3))
          b = torch.stack((a,a,a))
          b = b.type(torch.float)
          print(conv == (F.conv2d(input=img.reshape(1,img.shape[0],img.shape[1],img.shape[2]), weight=b.reshape(1, 3, 3, 3), bias=None, stride=1, padding=0, dilation=1, groups=1)))
          


      def kernel(self):
          k1 = torch.tensor([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
          k2 = torch.tensor([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
          k3 = torch.tensor([[ 1, 1, 1], [1, 1, 1], [1, 1, 1]])
          k4 = torch.tensor([[-1, -1, -1, -1, -1], [-1, -1, -1, -1, -1], [0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]])
          k5 = torch.tensor([[-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1]])
          if self.mode == 'known':
            if self.o_channel == 1:
                return torch.reshape(k1, (1, k1.shape[0], k1.shape[1]))
            elif self.o_channel == 2:
                return torch.stack((k4,k5))
            else:
                return torch.stack((k1,k2,k3))
          elif self.mode == 'rand':
                return torch.rand((self.o_channel, self.kernel_size, self.kernel_size))
          else:
                return None

In [ ]:
if __name__ == '__main__':
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  # Part A
  conv2d = Conv2D(in_channel=3, o_channel=1, kernel_size=3, stride=1, mode='known')
  img = Image.open('/content/drive/MyDrive/DeepLearning/BME595/ironman1280720.jpg')
  conv = conv2d.forward(input_image = img)
  #print(conv2d.kernel().shape,'\n\n', conv2d.kernel())
  #a = torch.reshape(conv2d.kernel(),(1,conv2d.kernel().shape[0],conv2d.kernel().shape[1]))

In [241]:
k1 = torch.tensor([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
k2 = torch.tensor([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
k3 = torch.tensor([[ 1, 1, 1], [1, 1, 1], [1, 1, 1]])
k4 = torch.tensor([[-1, -1, -1, -1, -1], [-1, -1, -1, -1, -1], [0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]])
k5 = torch.tensor([[-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1], [-1, -1, 0, 1, 1]])

k1 = torch.reshape(k1, (1, k1.shape[0], k1.shape[1]))
k1 = k1.view(1,k1.shape[0], k1.shape[1], k1.shape[2])
print(k1.shape)
print(conv.shape)
transform = transforms.Compose([transforms.ToTensor()])
img_trans = transform(img)
#img_trans = img_trans.view(1, img_trans.shape[0],img_trans.shape[1],img_trans.shape[2])
#print(F.conv2d(input=img_trans, weight=k1))
#print((conv == F.conv2d(input=img_trans, weight=k1)).sum() == conv.shape[0]*conv.shape[1]*conv.shape[2])
total = torch.zeros((1,718,1278))
k1 = k1.type(torch.float)
for i in range(img_trans.shape[0]):
    z = img_trans[i].view(1, 1, img_trans[i].shape[0], img_trans[i].shape[1])
    total = total + F.conv2d(input = z, weight = k1)
total = total.reshape((1,718,1278))
print(conv == total)

torch.Size([1, 1, 3, 3])
torch.Size([1, 718, 1278])
tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms

#plt.imshow(img)
print(type(img))
trans = transforms.Compose([transforms.ToTensor()])
img_trans = trans(img)
print(img_trans.shape)
print(img_trans[0].shape)
#plt.imshow(np.transpose(img_trans, (1,2,0)))
plt.imshow(img_trans[2])

# width, height = img.size
# print(width, height)
#img_tensor = torch.Tensor(img).to_tensor()

In [ ]:
import torch

a = torch.tensor([[1,2,3],[4,5,6],[7,8,9],[10,11,12]])
b = torch.tensor([[1,2],[4,5]])

# (W – F + 2P) / S + 1
h = a.shape[0]
w = a.shape[1]
print(h,w)


print(a[:2,:2], '\n\n', b, '\n\n', a[:2,:2] * b, '\n\n', torch.matmul(a[:2,:2],b),'\n\n', a[:2,:2] @ b,'\n\n', torch.mm(a[:2,:2], b))

4 3
tensor([[1, 2],
        [4, 5]]) 

 tensor([[1, 2],
        [4, 5]]) 

 tensor([[ 1,  4],
        [16, 25]]) 

 tensor([[ 9, 12],
        [24, 33]]) 

 tensor([[ 9, 12],
        [24, 33]]) 

 tensor([[ 9, 12],
        [24, 33]])


In [ ]:
a = torch.rand((8,7))
b = -torch.ones((2,2))
s = 2
h = int((a.shape[0] - b.shape[0] +2*0)/s + 1)
w = int((a.shape[1] - b.shape[1] +2*0)/s + 1)
c = torch.zeros((h,w))

print(a, '\n\n', b)
print('\n\n', c)
for i in range(c.shape[0]):
    for j in range(c.shape[1]):
        c[i,j] = (a[i*s:i*s+b.shape[0],j*s:j*s+b.shape[1]]*b).sum()
print(c)

tensor([[0.7395, 0.7864, 0.6659, 0.6644, 0.6232, 0.4127, 0.8050],
        [0.0448, 0.6490, 0.0493, 0.8421, 0.9548, 0.6959, 0.6195],
        [0.1866, 0.0436, 0.5386, 0.1172, 0.3958, 0.8312, 0.8699],
        [0.2881, 0.5981, 0.8526, 0.6223, 0.1749, 0.6002, 0.3097],
        [0.0368, 0.1093, 0.5377, 0.7068, 0.7558, 0.8756, 0.1982],
        [0.1433, 0.3499, 0.1206, 0.4696, 0.6780, 0.3501, 0.8097],
        [0.7789, 0.5764, 0.2899, 0.2655, 0.9193, 0.3716, 0.2269],
        [0.7860, 0.1281, 0.5304, 0.9580, 0.3930, 0.6692, 0.9114]]) 

 tensor([[-1., -1.],
        [-1., -1.]])


 tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[-2.2197, -2.2218, -2.6865],
        [-1.1164, -2.1307, -2.0021],
        [-0.6393, -1.8347, -2.6595],
        [-2.2694, -2.0438, -2.3531]])


In [221]:
import torch.nn.functional as F

a = a.view(1,1,8,7)
b = b.view(1,1,2,2)
test = F.conv2d(input=a, weight=b, stride=2)
print(test)

print((c == test).sum() == c.shape[0] * c.shape[1])

tensor([[[[-2.2197, -2.2218, -2.6865],
          [-1.1164, -2.1307, -2.0021],
          [-0.6393, -1.8347, -2.6595],
          [-2.2694, -2.0438, -2.3531]]]])
tensor(True)
